### Tutorial for tokenizer

In [1]:
import os,sys
sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..')))
import warnings
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from SpatialGPT.config import config
opt = config()
args = opt.parse_args(['--gpu', '0'])
args.dataset_path = '/mnt/d/Dataset/SRT_Dataset/1-DLPFC/'
args.max_epoch = 30
if args.version.split('-')[0] != 'dev':
    warnings.filterwarnings("ignore")
args

Namespace(amp=True, att_mask=True, batch_size=32, clusters=0, d_model=100, dataset_path='/mnt/d/Dataset/SRT_Dataset/1-DLPFC/', decoder='NB', dropout=0.2, early_stop=True, eval=False, f_dim=512, fast=True, flow='source_to_target', gid_emb=50, gpu=0, hvgs=1200, latent_dim=64, log_file='../Log', log_steps=1, lr=0.0001, max_epoch=30, max_neighbors=6, n_gcn=3, n_head=4, n_layer=4, n_randomwalk=[5, 15], n_randwalk=50, n_spot=0, n_token=60697, out_file='../Output', p_randomwalk=[1, 1.5], project_name='SpatialGPT', q_randomwalk=[1, 0.5], schedule_ratio=0.9, seed=2024, srt_resolution=2, tau=1, version='dev-1.0', visual=True, vocab_path='/home/zl/SpatialGRN/Data/default_census_vocab.json', wegiht_decay=0)

In [3]:
from SpatialGPT.utils import set_random_seed
set_random_seed(args.seed)

**Read data**

In [4]:
sector = '151507'
adata = sc.read_visium(os.path.join(args.dataset_path+sector))
adata

/home/zl/anaconda3/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/zl/anaconda3/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

**Token generation**

In [6]:
from SpatialGPT.tokenizer import GeneID


In [13]:
from pathlib import Path
path = Path('/home/zl/SpatialGRN/Data/default_census_vocab.json')
import json
with path.open('r') as f:
    tokens = json.load(f)
vocab = GeneID.from_file(path)
'<pad>' in vocab, len(vocab)

(True, 60697)

In [46]:
adata.var["gene_name"] = adata.var.index.tolist()
adata.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata.var["gene_name"]
    ]
gene_ids_in_vocab = np.array(adata.var["id_in_vocab"])
np.sum(gene_ids_in_vocab >= 0)/len(gene_ids_in_vocab)

0.6814359830639871

In [52]:
adata

AnnData object with n_obs × n_vars = 4226 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'gene_name', 'id_in_vocab'
    uns: 'spatial'
    obsm: 'spatial'

In [9]:
import numpy as np
a=[5, 15]
for i in range(a[0], a[1], 2):
    print(i)

5
7
9
11
13


In [3]:
import torch

def mask_feature(x, mask_prob):
    """
    Args:
        x (torch.Tensor): 输入特征矩阵，形状为 (num_nodes, num_features)
        mask_prob (float): 每个特征被掩盖的概率
    Returns:
        torch.Tensor: 掩盖后的特征矩阵
    """
    # 生成与输入相同形状的掩码矩阵
    mask = torch.bernoulli(torch.full(x.shape, 1 - mask_prob)).to(x.device)
    # 应用掩码矩阵到输入特征
    x_masked = x * mask
    return x_masked


In [26]:
x = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
mask_prob = 0.1
x_masked = mask_feature(x, mask_prob)
print(x_masked)


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 0., 9.]])
